Initialise code for google colab

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_addons as tfa
print(tfa.__version__)

In [ ]:
!pip install tensorflow==2.11
!pip install tensorflow-addons==0.19

Create data base files under google colab environment

In [ ]:
!unzip -q '/content/drive/MyDrive/data_equalize.zip' -d '/content/'

For Luc because my archive made on mac create a __MACOSX folder

In [ ]:
%rm -rf /content/__MACOSX

Define working directory to our jupyter repertory:
* because path to the different repertories (./data, ./output...) are define relatevly to jupyter one
* let import _mypath which add ./lib to python path in order to import our own define libraries


In [ ]:
# for google colab use
%cd /content/gdrive/MyDrive/covid-19-xRay/jupyter
db_work_dir = '/content'

In [1]:
# for local use
db_work_dir = '..'

In [27]:
import _mypath
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications import DenseNet201
import joblib

%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
%aimport database.data_path
%aimport database.dataset

from database.data_path import build_data_paths
from database.dataset import build_dataset


Model parameters

In [20]:
batch_size = 4
image_height = 256
image_width = 256
image_size = [image_height, image_width]
input_shape = (image_height, image_width, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 100

label_smoothing = 0.1
patience = 5
min_delta = 0.005

# data augmentation
scale = 1. / 255.
flip = "horizontal_and_vertical"
rotation_factor = 10. / 360.
zoom_height_factor = 0.2
zoom_width_factor = 0.2

mlp_head_units = [512, 256]  # Size of the dense layers of the final classifier
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

Build paths and variables for reading data base hierarchy

In [26]:
# input
db_name = 'data'
db_path = os.path.join(db_work_dir, db_name)

# output
output_path = os.path.join('..', 'output', 'transfer_learning')
if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)

log_path = os.path.join(output_path, 'log')
if not os.path.exists(log_path):
    os.makedirs(log_path, exist_ok=True)

ckpt_path = os.path.join(output_path, 'ckpt')
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path, exist_ok=True)

metric_path = os.path.join(output_path, 'metric')
if not os.path.exists(metric_path):
    os.makedirs(metric_path, exist_ok=True)


Structure to manage paths in data base

In [45]:
grain_types = ["hard_wheat", "soft_wheat", "barley", "corn"]
n_labels = len(grain_types)
print(n_labels)

data_paths_device1, labels_device1 = build_data_paths(db_path, grain_types, ["device1"])
print(data_paths_device1)
data_paths_device2, labels_device2 = build_data_paths(db_path, grain_types, ["device2"])
print(data_paths_device2)


4
['../data/hard_wheat_device1', '../data/soft_wheat_device1', '../data/barley_device1', '../data/corn_device1']
['../data/soft_wheat_device2', '../data/barley_device2']


Create tf Dataset

In [76]:
ds_device1 = build_dataset(data_paths_device1, labels_device1, image_size=image_size, shuffle=False)
print(ds_device1)
print(labels_device1)

ds_device2 = build_dataset(data_paths_device2, labels_device2, image_size=image_size, shuffle=False)
print(ds_device2)
print(labels_device2)

Found 26 files belonging to 1 classes.


Found 26 files belonging to 1 classes.
Found 51 files belonging to 1 classes.
Found 51 files belonging to 1 classes.
<_ConcatenateDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.int32, name=None))>
[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
Found 11 files belonging to 1 classes.
Found 11 files belonging to 1 classes.
<_ConcatenateDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.int32, name=None))>
[[0, 1, 0, 0], [0, 0, 1, 0]]


In [78]:
dataset_size = len(ds_device1)
print(dataset_size)
train_size = int(0.8 * dataset_size)
valid_size = int(0.10 * dataset_size)

ds = ds_device1.shuffle(dataset_size, seed=123, reshuffle_each_iteration=False)
# ds = ds_device1
ds_train = ds.take(train_size)
ds_test = ds.skip(train_size)
ds_valid = ds_test.take(valid_size)
ds_test = ds_test.skip(valid_size)

# for elem in ds_test:
#     print(elem[1])
# print("-----------")
# for elem in ds_test:
#     print(elem[1])

154


2023-06-20 14:14:20.507926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [26]
	 [[{{node Placeholder/_0}}]]
2023-06-20 14:14:20.508499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [26]
	 [[{{node Placeholder/_0}}]]


tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 1 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([0 0 1 0], shape=(4,), dtype=int32)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 1 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int32)
-----------
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 1 0], shape=(4,), dtype=int32)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int32)
tf.Tensor([1 0 0 0], s

Data augmentation

In [51]:
data_augmentation = keras.Sequential(
    [
      layers.Rescaling(scale=scale),
      layers.RandomFlip(flip),
      layers.RandomRotation(rotation_factor),
      layers.RandomZoom(height_factor=zoom_height_factor, width_factor=zoom_width_factor),
    ],
    name='data_augmentation'
)
norm_layer = layers.Normalization()


Pretrained model

In [52]:
# preproc = tf.keras.applications.vgg16.preprocess_input(augmented)
baseModel = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
# preproc = tf.keras.applications.vgg19.preprocess_input(input)
# baseModel = VGG19(weights="imagenet", include_top=False,input_tensor=layers.Input(shape=(image_size, image_size, 3)))
# preproc = tf.keras.applications.DenseNet201.preprocess_input(input)
# baseModel = DenseNet201(weights="imagenet", include_top=False,input_tensor=layers.Input(shape=(image_size, image_size, 3)))
for layer in baseModel.layers:
    layer.trainable = False


inputs = layers.Input(shape=input_shape)
augmented = data_augmentation(inputs)
base = baseModel(augmented)
# construct the head of the model that will be placed on top of the
head = layers.AveragePooling2D(pool_size=(8, 8))(base)
head = layers.Flatten(name="flatten")(head)
features = mlp(head, hidden_units=mlp_head_units, dropout_rate=0.5)

# Classify outputs.
softmax = layers.Dense(n_labels, activation='softmax', kernel_initializer='random_normal')(features)

# Create the Keras model.
model = keras.Model(inputs=inputs, outputs=softmax)


In [53]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 input_8 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 256, 256, 3)      0         
 al)                                                             
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 average_pooling2d_3 (Averag  (None, 1, 1, 512)        0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
          

In [79]:
from run_exp.standard import run_experiment

model_name = 'transfer_leraning_VGG16'
run_experiment(
  model,
  ds_train, ds_valid, ds_test,
  batch_size=batch_size, num_epochs=num_epochs,
  learning_rate=learning_rate, weight_decay=weight_decay,
  from_logits=False, label_smoothing=label_smoothing,
  patience=patience, min_delta=min_delta,
  log_path=log_path, ckpt_path=ckpt_path,
  prefix=model_name
)


Epoch 1/100


2023-06-20 14:15:02.436797: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_23' with dtype string and shape [51]
	 [[{{node Placeholder/_23}}]]
2023-06-20 14:15:02.437328: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [26]
	 [[{{node Placeholder/_0}}]]


31/31 [==============================] - ETA: 0s - loss: 0.7142 - accuracy: 0.8049

2023-06-20 14:15:50.484746: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_15' with dtype string and shape [51]
	 [[{{node Placeholder/_15}}]]
2023-06-20 14:15:50.485518: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_15' with dtype string and shape [51]
	 [[{{node Placeholder/_15}}]]



Epoch 1: val_accuracy improved from -inf to 0.80000, saving model to ../output/transfer_learning/ckpt/transfer_leraning_VGG16_weights.hdf5
31/31 [==============================] - 62s 2s/step - loss: 0.7142 - accuracy: 0.8049 - val_loss: 0.7284 - val_accuracy: 0.8000
Epoch 2/100
31/31 [==============================] - ETA: 0s - loss: 0.6215 - accuracy: 0.8699
Epoch 2: val_accuracy improved from 0.80000 to 1.00000, saving model to ../output/transfer_learning/ckpt/transfer_leraning_VGG16_weights.hdf5
31/31 [==============================] - 63s 2s/step - loss: 0.6215 - accuracy: 0.8699 - val_loss: 0.6088 - val_accuracy: 1.0000
Epoch 3/100
31/31 [==============================] - ETA: 0s - loss: 0.5656 - accuracy: 0.9187
Epoch 3: val_accuracy did not improve from 1.00000
31/31 [==============================] - 73s 2s/step - loss: 0.5656 - accuracy: 0.9187 - val_loss: 0.5210 - val_accuracy: 1.0000
Epoch 4/100
31/31 [==============================] - ETA: 0s - loss: 0.5614 - accuracy: 0.

In [80]:
# model_name = 'transfer_leraning_VGG16'
checkpoint_filename = os.path.join(ckpt_path, model_name + '_weights.hdf5')
model.load_weights(checkpoint_filename)


In [85]:
%aimport run_exp.test
from run_exp.test import compile_test_model

y_test_pd, y_pred_pd, accuracy, conf_mat, report = compile_test_model(
    model,
    ds_test,
    batch_size,
    from_logits=False,
    label_smoothing=label_smoothing
)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

print(report)
f_name = os.path.join(metric_path, model_name + '_report.txt')
with open(f_name, "w") as text_file:
  text_file.write(report)

conf_mat
f_name = os.path.join(metric_path, model_name + '_conf_mat.joblib')
joblib.dump(conf_mat, f_name)


2023-06-20 14:28:23.861316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_15' with dtype string and shape [51]
	 [[{{node Placeholder/_15}}]]
2023-06-20 14:28:23.862457: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_23' with dtype string and shape [51]
	 [[{{node Placeholder/_23}}]]


4/4 [==============================] - 15s 1s/step


2023-06-20 14:28:38.906318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [26]
	 [[{{node Placeholder/_0}}]]
2023-06-20 14:28:38.906881: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_23' with dtype string and shape [51]
	 [[{{node Placeholder/_23}}]]


Test accuracy: 100.0%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         6

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



['../output/transfer_learning/metric/transfer_leraning_VGG16_conf_mat.joblib']

In [86]:
%aimport run_exp.test
from run_exp.test import compile_test_model

y_test_pd, y_pred_pd, accuracy, conf_mat, report = compile_test_model(
    model,
    ds_device2, batch_size,
    from_logits=False,
    label_smoothing=label_smoothing
)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

print(report)
f_name = os.path.join(metric_path, model_name + '_device2_report.txt')
with open(f_name, "w") as text_file:
  text_file.write(report)

conf_mat
f_name = os.path.join(metric_path, model_name + '_device2_conf_mat.joblib')
joblib.dump(conf_mat, f_name)


2023-06-20 14:28:57.053607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype string and shape [11]
	 [[{{node Placeholder/_7}}]]
2023-06-20 14:28:57.053909: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [11]
	 [[{{node Placeholder/_0}}]]


6/6 [==============================] - 7s 1s/step


2023-06-20 14:29:04.522972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [11]
	 [[{{node Placeholder/_0}}]]
2023-06-20 14:29:04.523322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [11]
	 [[{{node Placeholder/_0}}]]


Test accuracy: 50.0%
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        11
           2       0.50      1.00      0.67        11

    accuracy                           0.50        22
   macro avg       0.25      0.50      0.33        22
weighted avg       0.25      0.50      0.33        22



/Users/luc/.local/share/virtualenvs/inarix-PNHHY2Cu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/luc/.local/share/virtualenvs/inarix-PNHHY2Cu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/luc/.local/share/virtualenvs/inarix-PNHHY2Cu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

['../output/transfer_learning/metric/transfer_leraning_VGG16_device2_conf_mat.joblib']